In [1]:
import os

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cobra
from cobra.io import read_sbml_model, write_sbml_model
from cobra import Reaction, Metabolite
from cobra.flux_analysis import pfba
from tqdm.auto import tqdm

In [2]:
model_dir = './iML1515.xml'
output_dir = './model_information'
model_output_dir = './model_files'

if output_dir[-1] == '/':
    output_dir = output_dir[:-1]
if model_output_dir[-1] == '/':
    model_output_dir = model_output_dir[:-1]

if not os.path.exists(output_dir):
    os.makedirs(output_dir)
if not os.path.exists(model_output_dir):
    os.makedirs(model_output_dir)

In [3]:
base = read_sbml_model(model_dir)
base

Academic license - for non-commercial use only - expires 2024-06-11
Using license file /data/user_home/gbkim/gurobi.lic


Name,iML1515
Memory address,0x07fb4bd448130
Number of metabolites,1877
Number of reactions,2712
Number of groups,38
Objective expression,1.0*BIOMASS_Ec_iML1515_core_75p37M - 1.0*BIOMASS_Ec_iML1515_core_75p37M_reverse_35685
Compartments,"cytosol, extracellular space, periplasm"


In [4]:
cinmcoa_c = Metabolite('cinmcoa_c', name='Cinnamoyl-CoA', formula='C30H38N7O17P3S', charge=-4, compartment='c')
hppncoa_c = Metabolite('3hppncoa_c', name='3-Hydroxy-3-phenylpropanoyl-CoA', formula='C30H40N7O18P3S', charge=-4, compartment='c')
kppcoa_c = Metabolite('kppcoa_c', name='3-Oxo-3-phenylpropanoyl-CoA', formula='C30H38N7O18P3S', charge=-4, compartment='c')
bzal_c = Metabolite('bzal_c', name='Benzaldehyde', formula='C7H6O', charge=0, compartment='c')
bz_c = Metabolite('bz_c', name='Benzoate', formula='C7H5O2', charge=-1, compartment='c')
mndlt__S_c = Metabolite('mndlt__S_c', name='(S)-Mandelate', formula='C8H7O3', charge=-1, compartment='c')
phglx_c = Metabolite('phglx_c', name='Phenylglyoxylate', formula='C8H5O3', charge=-1, compartment='c')
bzalc_c = Metabolite('bzalc_c', name='Benzyl alcohol', formula='C7H8O', charge=0, compartment='c')
bzac_c = Metabolite('bzac_c', name='Benzyl acetate', formula='C9H10O2', charge=0, compartment='c')


bz_e = Metabolite('bz_e', name='Benzoate', formula='C7H5O2', charge=-1, compartment='e')
bzalc_e = Metabolite('bzalc_e', name='Benzyl alcohol', formula='C7H8O', charge=0, compartment='e')
bzac_e = Metabolite('bzac_e', name='Benzyl acetate', formula='C9H10O2', charge=0, compartment='e')

In [5]:
PAL = Reaction('PAL', name='phenylalanine lyase', lower_bound=0, upper_bound=1000)
PAL.add_metabolites({
    base.metabolites.phe__L_c: -1, base.metabolites.cinnm_c: 1, base.metabolites.nh4_c: 1,
})

CNL = Reaction('CNL', name='cinnamate:CoA ligase', lower_bound=0, upper_bound=1000)
CNL.add_metabolites({
    base.metabolites.cinnm_c: -1, base.metabolites.atp_c: -1, base.metabolites.coa_c: -1,
    cinmcoa_c: 1, base.metabolites.amp_c: 1, base.metabolites.ppi_c: 1,
})

ENCL = Reaction('ENCL', name='cinnamoyl-CoA hydratase', lower_bound=-1000, upper_bound=1000)
ENCL.add_metabolites({
    cinmcoa_c: -1, base.metabolites.h2o_c: -1, hppncoa_c: 1
})

AIM1 = Reaction('AIM1', name='3-hydroxyacyl-CoA dehydrogenase', lower_bound=0, upper_bound=1000)
AIM1.add_metabolites({
    hppncoa_c: -1, base.metabolites.nad_c: -1,
    kppcoa_c: 1, base.metabolites.nadh_c: 1, base.metabolites.h_c: 1
})

PHDC = Reaction('PHDC', name='3-oxoacyl-CoA ketohydrolase', lower_bound=0, upper_bound=1000)
PHDC.add_metabolites({
    kppcoa_c: -1, base.metabolites.h2o_c: -1, 
    bz_c: 1, base.metabolites.accoa_c: 1, base.metabolites.h_c: 1
})

CAR1 = Reaction('CAR1', name='carboxylic acid reductase', lower_bound=0, upper_bound=1000)
CAR1.add_metabolites({
    bz_c: -1, base.metabolites.atp_c: -1, base.metabolites.h_c: -1, base.metabolites.nadph_c: -1,
    bzal_c: 1, base.metabolites.amp_c: 1, base.metabolites.ppi_c: 1, base.metabolites.nadp_c: 1,
})


BZALDH2 = Reaction('BZALDH2', name='benzyl alcohol dehydrogenase (NADPH)', lower_bound=-1000, upper_bound=1000)
BZALDH2.add_metabolites({
    bzal_c: -1, base.metabolites.nadph_c: -1, base.metabolites.h_c: -1,
    bzalc_c: 1, base.metabolites.nadp_c: 1
})

ATF1 = Reaction('ATF1', name='alcohol acetyltransferase', lower_bound=0, upper_bound=1000)
ATF1.add_metabolites({
    bzalc_c: -1, base.metabolites.accoa_c: -1,
    bzac_c: 1, base.metabolites.coa_c: 1
})



HMAS = Reaction('HMAS', name='hydroxymandelate synthase', lower_bound=0, upper_bound=1000)
HMAS.add_metabolites({
    base.metabolites.phpyr_c: -1, base.metabolites.o2_c: -1,
    mndlt__S_c: 1, base.metabolites.co2_c: 1
})


MDLB3 = Reaction('MDLB3', name='(S)-mandelate dehydrogenase (FMN)', lower_bound=0, upper_bound=1000)
MDLB3.add_metabolites({
    mndlt__S_c: -1, base.metabolites.fmn_c: -1,
    phglx_c: 1, base.metabolites.fmnh2_c: 1, 
})


MDLC = Reaction('MDLC', name='benzolformate decarboxylase', lower_bound=0, upper_bound=1000)
MDLC.add_metabolites({
    phglx_c: -1, base.metabolites.h_c: -1,
    bzal_c: 1, base.metabolites.co2_c: 1
})


BZtex = Reaction('BZtex', name='benzoate to extracellular space', lower_bound=0, upper_bound=1000)
BZtex.add_metabolites({
    bz_c: -1, bz_e: 1
})


BZALCtex = Reaction('BZALCtex', name='benzyl alcohol to extracellular space', lower_bound=0, upper_bound=1000)
BZALCtex.add_metabolites({
    bzalc_c: -1, bzalc_e: 1
})


BZAZtex = Reaction('BZAZtex', name='benzyl acetate to extracellular space', lower_bound=0, upper_bound=1000)
BZAZtex.add_metabolites({
    bzac_c: -1, bzac_e: 1
})


EX_bz_e = Reaction('EX_bz_e', name='benzoate exchange reaction', lower_bound=0, upper_bound=1000)
EX_bz_e.add_metabolites({
    bz_e: -1
})

EX_bzalc_e = Reaction('EX_bzalc_e', name='benzyl alcohol exchange reaction', lower_bound=0, upper_bound=1000)
EX_bzalc_e.add_metabolites({
    bzalc_e: -1
})

EX_bzac_e = Reaction('EX_bzac_e', name='benzyl acetate exchange reaction', lower_bound=0, upper_bound=1000)
EX_bzac_e.add_metabolites({
    bzac_e: -1
})

In [6]:
new_metabolites = [
    cinmcoa_c, hppncoa_c, kppcoa_c,  
    bzal_c, bz_c, mndlt__S_c, phglx_c, 
    bzalc_c, bzac_c,
    bz_e, bzalc_e, bzac_e,
]

In [7]:
new_reactions = [
    PAL, CNL, ENCL, AIM1, PHDC,
    CAR1, BZALDH2, ATF1,
    HMAS, MDLB3, MDLC,
    BZtex, EX_bz_e, BZALCtex, 
    EX_bzalc_e, BZAZtex, EX_bzac_e,
]

In [8]:
met_info = []
for met in new_metabolites:
    met_id = met.id
    met_name = met.name
    formula = met.formula
    charge = met.charge
    comp = met.compartment
    met_info.append([met_id, met_name, formula, charge, comp])
df_met = pd.DataFrame(met_info)
df_met.columns = ['ID', 'Name', 'Formula', 'Charge', 'Compartment']
df_met.to_csv(f'{output_dir}/Added_metabolites.txt', sep='\t')

In [9]:
rxn_info = []
for rxn in new_reactions:
    rxn_id = rxn.id
    rxn_name = rxn.name
    lb, ub = rxn.bounds
    reaction = rxn.reaction
    rxn_info.append([rxn_id, rxn_name, reaction, lb, ub])
df_rxn = pd.DataFrame(rxn_info)
df_rxn.columns = ['ID', 'Name', 'Reaction', 'Lower bound', 'Upper_bound']
df_rxn.to_csv(f'{output_dir}/Added_reactions.txt', sep='\t')

### Construct GEMs

#### all pathways in integrated strains

In [10]:
# Benzoic acid dependent pathway
bz_dp = [
    PAL, CNL, ENCL, AIM1, 
    PHDC, CAR1, BZALDH2, ATF1,
    BZAZtex, EX_bzac_e
]


# Benzoic acid independent pathway
bz_idp = [
    HMAS, MDLB3, MDLC, BZALDH2, ATF1,
    BZAZtex, EX_bzac_e
]

In [11]:
with base as m:
    m.add_reactions(bz_dp)
    write_sbml_model(m, f'{model_output_dir}/iML1515_bz_dp.xml')

with base as m:
    m.add_reactions(bz_idp)
    write_sbml_model(m, f'{model_output_dir}/iML1515_bz_idp.xml')

#### Using co-culture

In [12]:
# Benzoic acid dependent pathway Strain 1
bz_dp_co_1 = [
    PAL, CNL, ENCL, AIM1, PHDC,
    BZtex, EX_bz_e,
]

# Benzoic acid dependent pathway Strain 2
bz_dp_co_2 = [
    BZtex, EX_bz_e,
    CAR1, BZALDH2, ATF1,
    BZAZtex, EX_bzac_e
]

# Benzoic acid independent pathway Strain 1
bz_idp_co_1 = [
    HMAS, MDLB3, MDLC, BZALDH2, 
    BZALCtex, EX_bzalc_e,
]

# Benzoic acid independent pathway Strain 2
bz_idp_co_2 = [
    BZALCtex, EX_bzalc_e,
    ATF1,
    BZAZtex, EX_bzac_e
]

In [13]:
with base as m:
    m.add_reactions(bz_dp_co_1)
    write_sbml_model(m, f'{model_output_dir}/iML1515_bz_dp_co_1.xml')

with base as m:
    m.add_reactions(bz_dp_co_2)
    m.reactions.BZtex.bounds = (-1000, 1000)
    m.reactions.EX_bz_e.bounds = (-1000, 1000)
    write_sbml_model(m, f'{model_output_dir}/iML1515_bz_dp_co_2.xml')
    
    
with base as m:
    m.add_reactions(bz_idp_co_1)
    write_sbml_model(m, f'{model_output_dir}/iML1515_bz_idp_co_1.xml')
    
with base as m:
    m.add_reactions(bz_idp_co_2)
    m.reactions.BZALCtex.bounds = (-1000, 1000)
    m.reactions.EX_bzalc_e.bounds = (-1000, 1000)
    write_sbml_model(m, f'{model_output_dir}/iML1515_bz_idp_co_2.xml')

In [14]:
model_desc = [
    'Integrated strain-benzoic acid dependent pathway', 
    'Integrated strain-benzoic acid independent pathway', 
    'Co-culture strain-benzoic acid dependent pathway Strain 1',
    'Co-culture strain-benzoic acid dependent pathway Strain 2',
    'Co-culture strain-benzoic acid independent pathway Strain 1',
    'Co-culture strain-benzoic acid independent pathway Strain 2'
]
added_reactions = [
    bz_dp,
    bz_idp,
    bz_dp_co_1,
    bz_dp_co_2,
    bz_idp_co_1,
    bz_idp_co_2
]

with open(f'{output_dir}/Added_reactions_per_model.txt', 'w') as fp:
    fp.write('Description\tReactionIDs\n')
    for desc, rxns in zip(model_desc, added_reactions):
        rxn_ids = '\t'.join([rxn.id for rxn in rxns])
        fp.write(f'{desc}\t{rxn_ids}\n')